In [1]:
# repo_dir = "Repos"  
# %load_ext autoreload
# %autoreload 2

# import sys
# from pathlib import Path
# sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
# sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-vgg19" / "src" / "vgg19"))
# sys.path.insert(0, str(Path.home() / repo_dir / "RETFound_MAE"))
# sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-exec" / "models" / "vgg19")) 

In [14]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
import random

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration, DatasetVersion
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [2]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'



gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

You are already logged in.


In [23]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-04-12 13:17:59,012 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install


In [24]:
datasets = ['5-XW4J'] # a image dataset of multimodal for testing

to_be_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    to_be_download.append(ds_dict)

workflow_instance = EA.create_workflow(
    name="Feature Ingestion",
    workflow_type="Test Workflow"
)

config = ExecutionConfiguration(
    datasets=to_be_download,
    assets = ['2-4JR6'],
    workflow=workflow_instance,
    description="Test of feature asset with bounding box")

execution = EA.create_execution(config)

2025-04-12 13:18:22,663 - WARNING - File /home/lizhiwei/Repos/eye-ai-exec/notebooks/templates/crop_multimodal_images_test.ipynb has been modified since last commit. Consider commiting before executing
2025-04-12 13:18:23,224 - INFO - Materialize bag 5-XW4J... 
2025-04-12 13:18:23,964 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-04-12 13:18:24,975 - INFO - Loading /data/5-XW4J_01888e4042f24555eb76b5162749507ee5893239d85cd51eec8a9e73409a9c4a/Dataset_5-XW4J
2025-04-12 13:18:25,279 - INFO - Creating new database for dataset: 5-XW4J in /data/lizhiwei/EyeAI_working/5-XW4J@334-PTGP-NSB2.db
2025-04-12 13:18:25,337 - INFO - Downloading assets ...
2025-04-12 13:18:31,226 - INFO - File [/data/lizhiwei/EyeAI_working/deriva-ml/execution/5-XXHW/downloaded-assets/Execution_Asset/optic_disk_crop_model.hdf5] transfer successful. 76.75 MB transferred at 14.19 MB/second. Elapsed time: 0:00:05.410616.
2025-04-12 13:18:31,859 - INFO - Initialize status finishe

In [25]:
print(execution)

caching_dir: /data
_working_dir: /data/lizhiwei/EyeAI_working
execution_rid: 5-XXHW
workflow_rid: 5-XXC6
asset_paths: {'Execution_Asset': [AssetFilePath('/data/lizhiwei/EyeAI_working/deriva-ml/execution/5-XXHW/downloaded-assets/Execution_Asset/optic_disk_crop_model.hdf5')]}
configuration: datasets=[DatasetSpec(rid='5-XW4J', materialize=True, version=DatasetVersion(major=0, minor=2, patch=0))] assets=['2-4JR6'] workflow=Workflow(name='Feature Ingestion', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob//notebooks/templates/crop_multimodal_images_test.ipynb', workflow_type='Test Workflow', version=None, description='', rid=None, checksum='e69de29bb2d1d6434b8b29ae775ad8c2e48c5391') parameters={} description='Test of feature asset with bounding box' argv=['/home/lizhiwei/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/lizhiwei/.local/share/jupyter/runtime/kernel-b1c468f9-2c01-4bbc-b327-33c7a8e6b74b.json']


In [26]:
# training_ds_bag = execution.datasets[0]
# testing_ds_bag = execution.datasets[1]
# multimodal_full_ds_bag = execution.datasets[2]
ds_bag = execution.datasets[0]
image_rids = [d["RID"] for d in execution.datasets[0].list_dataset_members()["Image"]]
crop_image_model = execution.asset_paths['Execution_Asset'][0]

In [27]:
ImageAnnotationFeature = EA.feature_record_class("Image", "Annotation")
ImageAnnotationFeature

deriva_ml.feature.ImageFeatureAnnotation

In [28]:
bounding_box_files = []
(EA.working_dir / "scratch").mkdir(exist_ok=True)
for image_rid in image_rids:
    fn = f"{EA.working_dir}/scratch/box{image_rid}.txt"
    with open(fn, "w") as fp:
        fp.write(f"Hi there {image_rid} {random.random()}")
    bounding_box_files.append(fn)

In [29]:
annotation_feature_list = []
for image_rid, bb_file in zip(image_rids, bounding_box_files):
    bounding_box_file = execution.asset_file_path(
        "Fundus_Bounding_Box", bb_file
    )
    annotation_feature_list.append(
        ImageAnnotationFeature(
            Image=image_rid,
            Fundus_Bounding_Box=bounding_box_file,
            Annotation_Function="Raw_Cropped_to_Eye",
            Annotation_Type="Optic Nerve",
        )
    )

In [30]:
with execution.execute() as execution:
    print(f"assets = {execution.asset_paths}")
    print(f"datasets = {execution.datasets}")
    execution.add_features(annotation_feature_list)

2025-04-12 13:19:17,834 - INFO - Start execution  ...
2025-04-12 13:19:17,883 - INFO - Start execution  ...
2025-04-12 13:19:17,924 - INFO - Successfully run Ml.
2025-04-12 13:19:17,960 - INFO - Algorithm execution ended.


assets = {'Execution_Asset': [AssetFilePath('/data/lizhiwei/EyeAI_working/deriva-ml/execution/5-XXHW/downloaded-assets/Execution_Asset/optic_disk_crop_model.hdf5')]}
datasets = [<deriva_ml.DatasetBag object 5-XW4J at 0x7f24cd9491b0>]


In [31]:


out = execution.upload_execution_outputs()

2025-04-12 13:19:19,803 - INFO - Uploading execution files...
2025-04-12 13:19:19,855 - INFO - Initializing uploader: GenericUploader v1.7.7 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2025-04-12 13:19:19,891 - INFO - Scanning files in directory [/data/lizhiwei/EyeAI_working/deriva-ml/execution/5-XXHW/asset]...
2025-04-12 13:19:19,893 - INFO - Including file: [/data/lizhiwei/EyeAI_working/deriva-ml/execution/5-XXHW/asset/deriva-ml/Execution_Metadata/configuration.json].
2025-04-12 13:19:19,893 - INFO - Including file: [/data/lizhiwei/EyeAI_working/deriva-ml/execution/5-XXHW/asset/deriva-ml/Execution_Metadata/environment_snapshot_20250412_131831.txt].
2025-04-12 13:19:19,894 - INFO - Including file: [/data/lizhiwei/EyeAI_working/deriva-ml/execution/5-XXHW/asset/eye-ai/Fundus_Bounding_Box/box2-BDAM.txt].
2025-04-12 13:19:19,895 - INFO - Including file: [/data/lizhiwei/EyeAI_working/deriva-ml/execution/5-XXHW/asset/eye-ai/Fundus_Bounding_Box/box2-BDB8.txt].

DerivaMLException: Fail to upload execution_assets. Error: [RuntimeError] 1 file(s) failed to upload due to errors.